# Task Description

In [1]:
# The task would be to test a long-only stock-based carry/value strategy.

# Carry strategy in stocks would be to long high dividend yield stocks and short the reverse (in this case no need to do short).
# Whole value strategy you should be familiar with.

# Carry trade in rate hike periods work pretty well (for fx especially). We are interested in whether the same hold for value. 
# The underlying assets would be S&P 500 stocks.

# Please take into account that some companies do stock repurchase instead of cash dividend.
# You could try to find adjusted dividend yield data. Time horizon is up to you, but keep in mind potential survivorship bias.
# It might be interesting to check the strategy performance during different periods. 
# Remember to make the backtest a way that they could be implemented (not longing 100 stocks at the same time). 
# Have a nice night and good luck with your presentation on Monday.

# Setup

## Load Packages

In [2]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import scipy as sp
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # to calculate the MSE
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from statsmodels.graphics.tsaplots import plot_acf # To produce ACF plots
from statsmodels.graphics.tsaplots import plot_pacf # To produce PACF plots
from statsmodels.tsa.seasonal import seasonal_decompose # To decompose Seasons
from statsmodels.tsa.stattools import adfuller, kpss # Tests for Stationarity
from statsmodels.tsa.ar_model import AutoReg # To produce AR models
from statsmodels.stats.anova import anova_lm # To use ANOVA (compare nested models)
from statsmodels.tsa.arima.model import ARIMA # To build ARMA & ARIMA Models
import statsmodels.stats.diagnostic as dg # To get Breusch-Godfrey Test
from statsmodels.stats.stattools import durbin_watson



from datetime import datetime # to transform variables into datetime objects
import math # simple math functions
from math import sqrt # square root function
import statistics # descriptive statistics library
import scipy.stats as stats # descriptive statistics library from scipy
import matplotlib.dates as mdates # date formatting
from matplotlib.collections import PolyCollection, LineCollection # better plot options

## Useful functions for Matrix calculations

In [3]:
def f(x, y): 
    return x + y                             
vecsum = np.vectorize(f)     # Use pd.DataFrame(vecsum(A,B)) to get df of elementwise sum

In [4]:
def g(x, y): 
    return x - y                             
vecdif = np.vectorize(g)     # Use pd.DataFrame(vecdif(A,B)) to get df of elementwise differences

In [5]:
def h(x, y): 
    return x * y                             
vecmult = np.vectorize(h)    # Use pd.DataFrame(vecmult(A,B)) to get df of elementwise multiplication

In [6]:
def i(x, y): 
    return x / y                             
vecdiv = np.vectorize(i)    # Use pd.DataFrame(vecdiv(A,B)) to get df of elementwise division

### Illustrative Example extra für de Herr Vogel

In [7]:
# Chasch de shit au lösche wenns checkt hesch aber mir hets na easy gholfe
A = [[1, 2, 3], 
    [3, 5, 6],
    [7, 8, 9]]
A = pd.DataFrame(A)
A.columns = ["ret1", "ret2", "ret3"]

B = [[0.2, 0.3, 0.5], 
    [0.1, 0.6, 0.3],
    [0.4, 0.2, 0.4]]
B = pd.DataFrame(B)    
B.columns = ["w1", "w2", "w3"]


In [8]:
A

,ret1,ret2,ret3
0,1,2,3
1,3,5,6
2,7,8,9


In [9]:
B

,w1,w2,w3
0,0.2,0.3,0.5
1,0.1,0.6,0.3
2,0.4,0.2,0.4


In [10]:
pd.DataFrame(vecsum(A,B))

,0,1,2
0,1.2,2.3,3.5
1,3.1,5.6,6.3
2,7.4,8.2,9.4


In [11]:
pd.DataFrame(vecdif(A,B))

,0,1,2
0,0.8,1.7,2.5
1,2.9,4.4,5.7
2,6.6,7.8,8.6


In [12]:
pd.DataFrame(vecmult(A,B))

,0,1,2
0,0.2,0.6,1.5
1,0.3,3.0,1.8
2,2.8,1.6,3.6


In [13]:
pd.DataFrame(vecdiv(A,B))

,0,1,2
0,5.0,6.666667,6.0
1,30.0,8.333333,20.0
2,17.5,40.000000,22.5


## Plotstyle

In [14]:
# Seaborn plot style ticks to have nicer looking plots
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## LoadData

In [15]:
# Loading Data

data_px = pd.read_excel("02_Data_clean/SPX_px_clean.xlsx", parse_dates=["Date"])
data_px_yly = pd.read_excel("02_Data_clean/SPX_px_clean_yearly.xlsx", parse_dates=["Date"])
data_bv = pd.read_excel("02_Data_clean/SPX_value_clean.xlsx", parse_dates=["Date"])

# Check
print(type(data_bv))
print(data_bv.dtypes)

<class 'pandas.core.frame.DataFrame'>
Date                  datetime64[ns]
AEE UN Equity                float64
ITT UN Equity                float64
UN UN Equity                 float64
APD UN Equity                float64
                           ...      
NTAP UQ Equity               float64
CTXS UQ Equity               float64
9990253D UN Equity           float64
0948669D UN Equity           float64
CMCSK UQ Equity              float64
Length: 501, dtype: object


In [16]:
data_px

,Date,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
0,1999-12-31,32.7500,5.5436,18.1458,31.0287,60.4375,17.6875,55.1812,23.5313,43.3125,...,67.6563,88.0625,13.8333,45.4688,35.7712,41.5313,48.9718,19.8750,23.6250,33.7083
1,2000-01-07,33.9375,5.6679,19.7500,33.7444,63.0625,17.3125,52.7163,26.6563,45.1250,...,48.2813,75.0000,16.8750,45.4375,27.1885,42.5000,44.4429,19.6250,22.1875,29.9583
2,2000-01-14,33.4375,5.6472,18.6875,29.9308,61.1875,16.9375,50.4754,25.9688,43.5000,...,49.5313,70.2188,17.2917,48.1875,25.9281,46.8438,53.9735,19.4375,23.5000,34.4167
3,2000-01-21,33.2500,5.4296,16.9792,26.7528,59.7500,17.1250,53.8367,23.6875,44.3438,...,53.5938,77.4688,15.7917,47.4375,24.7277,58.5625,59.5475,20.1875,22.6875,35.2500
4,2000-01-28,32.3125,5.2016,15.3333,27.5618,54.1250,16.3125,51.6519,21.7500,41.2813,...,52.0313,55.2813,14.3333,43.1875,21.2466,49.5938,53.6997,19.1250,22.3125,30.4167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2022-11-04,81.0300,75.6200,60.5000,276.0600,61.1600,14.2000,37.2400,227.8500,183.4200,...,124.2400,106.6900,10.2500,194.9200,10.3500,68.7900,103.9000,90.5400,18.9900,58.0000
1193,2022-11-11,83.3200,82.6000,60.5000,288.4600,61.1600,16.4400,38.3000,236.4900,186.4600,...,131.9200,121.4300,10.2500,194.9200,10.3500,73.5000,103.9000,90.5400,18.9900,58.0000
1194,2022-11-18,86.1800,84.2900,60.5000,296.4800,61.1600,15.6200,38.5500,231.4300,182.9900,...,123.3000,123.8500,10.2500,194.9200,10.3500,73.7200,103.9000,90.5400,18.9900,58.0000
1195,2022-11-25,88.8100,84.5800,60.5000,309.9100,61.1600,16.0200,39.0200,235.7000,183.7000,...,124.6300,123.4500,10.2500,194.9200,10.3500,74.6600,103.9000,90.5400,18.9900,58.0000


## Data Cleaning

In [17]:
# Resample monthly returns
data_px.set_index("Date", inplace=True) # set datetimeindex
data_bv.set_index("Date", inplace=True) # set datetimeindex
semian_ret = data_px.pct_change(fill_method = None).resample("2BQ").agg(lambda x: ((1+x).prod()-1)) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
# df = pd.concat([data, data2], axis=1) # concating dfs


In [18]:
# Drop first row as it was only to set the anchor for the resample function for December and June

# Sau müehsam dass wenns mehrmals renne lahsch immer wieder e row meh verlürsc
semian_ret = semian_ret.iloc[1: , :]

In [19]:
semian_ret

,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,KO UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-30,0.030534,-0.091601,-0.210104,-0.081937,0.018614,0.028269,-0.174619,-0.280214,-0.020922,-0.013947,...,-0.593534,-0.659333,-0.259034,0.815806,-0.721477,0.938297,-0.692074,0.223270,-0.195767,-0.199010
2000-12-29,0.372222,0.275726,0.463669,0.330629,-0.016244,-0.099381,-0.013529,0.396682,-0.004422,0.060936,...,0.221593,0.369793,-0.231707,-0.441332,-0.397587,-0.202640,0.188120,0.169666,-0.503289,0.030863
2001-06-29,-0.078003,0.141945,-0.053505,0.115854,-0.037853,-0.075092,0.067332,0.057858,0.071797,-0.261540,...,0.041115,-0.288457,0.353651,-0.105908,1.238391,-0.786563,0.551112,0.090110,0.165563,0.039521
2001-12-31,-0.007494,0.159320,-0.030554,0.026667,-0.145529,0.042904,-0.097757,0.062737,-0.000663,0.048222,...,-0.468906,-0.114227,0.200750,-0.005820,-0.142959,0.675182,-0.328080,-0.155806,0.270000,-0.161060
2002-06-28,0.014866,0.376214,0.122078,0.074514,0.110062,0.201899,-0.168222,-0.079714,-0.021451,0.187195,...,-0.538358,-0.469305,0.026039,-0.452927,-0.493747,-0.457952,-0.742430,-0.030187,0.192555,-0.345235
2002-12-31,-0.036736,-0.150563,-0.084875,-0.158906,-0.214583,-0.170616,-0.031132,-0.079469,-0.255367,-0.223750,...,-0.276968,0.355766,-0.079188,-0.056175,-0.293241,-0.130225,1.096016,-0.222222,-0.030612,-0.065852
2003-06-30,0.064205,0.076038,-0.089211,-0.032508,0.077171,0.117460,0.019537,0.239015,0.098331,0.069933,...,0.225806,-0.035686,0.020954,0.201228,0.344977,0.482440,0.635860,0.144883,0.223529,0.311180
2003-12-31,0.030619,0.144423,0.158121,0.273678,0.094525,0.277841,-0.144980,0.499373,0.158055,0.074178,...,0.305921,0.475515,0.415220,0.508455,0.043332,0.301122,0.043945,0.351770,0.069332,0.046233
2004-06-30,-0.036092,0.132429,0.085530,-0.005926,0.021493,-0.087594,0.054851,-0.066420,0.114173,0.002802,...,0.209068,0.320762,0.129724,-0.148071,0.172756,0.032583,-0.080943,0.019313,0.041174,-0.092308


### Create Book to Market Dataframe

In [20]:
data_px_yly

,Date,AEE UN Equity,ITT UN Equity,UN UN Equity,APD UN Equity,RDPL UN Equity,GOLD UN Equity,VZ UN Equity,CAT UN Equity,CVX UN Equity,...,PTC UQ Equity,QCOM UQ Equity,SPLS UQ Equity,XLNX UQ Equity,1519128D UQ Equity,NTAP UQ Equity,CTXS UQ Equity,9990253D UN Equity,0948669D UN Equity,CMCSK UQ Equity
0,1999-12-31,32.7500,5.5436,18.1458,31.0287,60.4375,17.6875,55.1812,23.5313,43.3125,...,67.6563,88.0625,13.8333,45.4688,35.7712,41.5313,48.9718,19.8750,23.6250,33.7083
1,2000-12-29,46.3125,6.4243,20.9792,37.9047,60.5625,16.3800,44.9293,23.6563,42.2188,...,33.5938,41.0938,7.8750,46.1250,6.0019,64.1875,17.9165,28.4375,9.4375,27.8333
2,2001-12-31,42.3000,8.3723,19.2033,43.3685,49.0200,15.9500,42.5405,26.1250,44.8050,...,19.5250,25.2500,12.4667,39.0500,11.3219,21.8700,18.0439,25.6400,13.7800,24.0000
3,2002-12-31,41.5700,10.0617,20.5700,39.5225,44.0200,15.4100,34.7334,22.8600,33.2400,...,6.3000,18.1950,12.2000,20.6000,4.6094,10.0000,9.8103,19.9300,15.8000,15.0600
4,2003-12-31,46.0000,12.3032,21.6333,48.8415,52.3900,22.7100,31.4438,41.5100,43.1950,...,9.8500,26.9650,18.2000,38.7400,5.8002,20.5300,16.8893,30.7400,21.8700,20.8533
5,2004-12-31,50.1400,14.0009,22.2367,53.5935,57.3800,24.2200,36.3109,48.7550,52.5100,...,14.7250,42.4000,22.4733,29.6500,6.2131,33.2200,19.5330,28.8400,25.3500,21.8933
6,2005-12-30,51.2400,17.0464,22.8833,54.7214,61.1600,27.8700,26.9979,57.7700,56.7700,...,15.2500,43.0800,22.7100,25.2100,8.6139,27.0000,22.9172,35.2700,21.9700,17.1267
7,2006-12-29,53.7300,18.8403,27.2500,64.9741,61.1600,30.7000,34.6415,61.3300,73.5300,...,18.0200,37.7900,26.7000,23.8100,7.9993,39.2800,21.5396,36.6400,23.8700,27.9200
8,2007-12-31,54.2100,21.8974,36.4600,91.1838,61.1600,42.0500,40.6414,72.5600,93.3300,...,17.8500,39.3500,23.0700,21.8700,8.5275,24.9600,30.2669,42.9000,20.3000,18.1200
9,2008-12-31,33.2600,15.2493,24.5500,46.4748,61.1600,36.7700,31.6758,44.6700,73.9700,...,12.6500,35.8300,17.9200,17.8200,6.4820,13.9700,18.7685,17.2400,7.5700,16.1500


In [21]:
data_px_yly.set_index("Date", inplace=True) # set datetimeindex

In [24]:
data_btm = pd.DataFrame(vecdiv(data_bv,data_px_yly))
data_btm

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.687548,2.255177,NaN,0.396526,NaN,0.593069,0.173405,0.328235,0.312175,0.066077,...,0.072443,0.041892,0.192825,0.120038,0.093066,0.036970,0.030054,NaN,0.821638,0.269750
1,0.503035,2.144498,NaN,0.360660,NaN,0.363339,0.279691,0.344678,0.367843,0.061525,...,0.109996,0.077946,0.321816,0.125596,0.620553,0.038123,0.191617,NaN,1.747709,0.370010
2,0.573714,1.850806,NaN,0.351160,NaN,0.373367,0.281501,0.312739,0.355083,0.096959,...,0.141373,0.137141,0.236743,0.145012,0.279635,0.117128,0.193733,NaN,1.027075,0.425333
3,0.599757,1.230955,NaN,0.432227,NaN,0.399176,0.342336,0.347664,0.445066,0.108978,...,0.290730,0.261550,0.307107,0.279335,0.755586,0.289830,0.372598,NaN,1.034823,1.851919
4,0.581046,1.627601,NaN,0.403004,NaN,0.287442,0.384540,0.212970,0.392958,0.113714,...,0.227858,0.061788,0.268775,0.184734,0.632461,0.192918,0.254445,NaN,0.859031,0.594079
5,0.592603,1.813283,NaN,0.384744,NaN,0.276338,0.373477,0.223296,0.408785,0.158895,...,0.199878,0.062788,0.245959,0.257508,0.628285,0.136246,0.277909,NaN,0.792525,0.570202
6,0.606741,1.730565,NaN,0.414191,NaN,0.256728,0.497176,0.217566,0.494487,0.171268,...,0.237882,0.086611,0.270167,0.315938,0.485216,0.189467,0.300080,NaN,0.920187,-0.051072
7,0.593030,1.661677,NaN,0.392752,NaN,0.535309,0.481218,0.173175,0.433117,0.151283,...,0.287625,0.254573,0.261678,0.251617,0.467053,0.137963,0.380439,NaN,0.846942,0.472658
8,0.597949,1.985222,NaN,0.263554,NaN,0.417018,0.432618,0.196195,0.395121,0.152853,...,0.340090,0.275367,0.351747,0.272506,0.415503,0.158766,0.324173,NaN,0.953148,0.641595
9,0.986106,2.208678,NaN,0.488058,NaN,0.476059,0.463524,0.226535,0.584460,0.195595,...,0.519225,0.339732,0.433901,0.396936,0.561879,0.384338,0.567520,NaN,1.724267,0.869474
